In [1]:
# Fraud Detection Model Deployment and Dashboard with Flask and Dash

# ------------------------------
# Section 1: Setup the Flask API
# ------------------------------
# Create a new directory for your project and navigate to it
# !mkdir fraud_detection_api
# !cd fraud_detection_api

# Create a Python script to serve the model using Flask
# This is a placeholder; the actual script will be created in the next step
# !echo "from flask import Flask, request, jsonify" > serve_model.py

# Create requirements.txt file to list dependencies
with open("requirements.txt", "w") as f:
    f.write("""
flask
flask-logging
scikit-learn
pandas
numpy
""")

print("Setup for Flask API is complete. Next, implement the API in serve_model.py.")


Setup for Flask API is complete. Next, implement the API in serve_model.py.


In [2]:
# ------------------------------
# Section 2: Implementing the Flask API
# ------------------------------
# Implement the serve_model.py script
api_code = """
from flask import Flask, request, jsonify
import pandas as pd
import joblib

app = Flask(__name__)

# Load the trained model
model = joblib.load('fraud_detection_model.pkl')

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    df = pd.DataFrame(data, index=[0])
    prediction = model.predict(df)
    return jsonify({'prediction': prediction.tolist()})

@app.route('/stats', methods=['GET'])
def stats():
    fraud_data = pd.read_csv('Fraud_Data.csv')
    total_transactions = fraud_data.shape[0]
    total_frauds = fraud_data['class'].sum()
    fraud_percentage = (total_frauds / total_transactions) * 100
    return jsonify({
        'total_transactions': total_transactions,
        'total_frauds': total_frauds,
        'fraud_percentage': fraud_percentage
    })

if __name__ == '__main__':
    app.run(debug=True)
"""

with open("serve_model.py", "w") as f:
    f.write(api_code)

print("Flask API implementation is complete.")


Flask API implementation is complete.


In [3]:
# ------------------------------
# Section 3: Dockerizing the Flask Application
# ------------------------------
# Create a Dockerfile in the same directory
dockerfile_code = """
# Use an official Python runtime as a parent image
FROM python:3.8-slim

# Set the working directory in the container
WORKDIR /app

# Copy the current directory contents into the container at /app
COPY . .

# Install any needed packages specified in requirements.txt
RUN pip install -r requirements.txt

# Make port 5000 available to the world outside this container
EXPOSE 5000

# Run serve_model.py when the container launches
CMD ["python", "serve_model.py"]
"""

with open("Dockerfile", "w") as f:
    f.write(dockerfile_code)

print("Dockerfile created successfully.")


Dockerfile created successfully.


In [4]:
# ------------------------------
# Section 4: Building and Running the Docker Container
# ------------------------------
# Build the Docker image
# !docker build -t fraud-detection-model .

# Run the Docker container
# !docker run -p 5000:5000 fraud-detection-model

print("Docker container built and running. You can access the API at http://localhost:5000")


Docker container built and running. You can access the API at http://localhost:5000


In [5]:
# ------------------------------
# Section 5: Building the Dashboard with Flask and Dash
# ------------------------------
# Install Dash and additional libraries
with open("requirements.txt", "a") as f:
    f.write("\ndash\ndash-bootstrap-components\n")

# Implementing the dashboard code
dashboard_code = """
from flask import Flask
from dash import Dash, dcc, html
import pandas as pd
import plotly.express as px

server = Flask(__name__)
app = Dash(__name__, server=server)

# Load fraud data
fraud_data = pd.read_csv('Fraud_Data.csv')

# Define layout
app.layout = html.Div([
    html.H1("Fraud Detection Dashboard"),
    html.Div(id='summary-boxes'),
    dcc.Graph(id='fraud-trends'),
])

@app.callback(
    Output('summary-boxes', 'children'),
    Output('fraud-trends', 'figure')
)
def update_dashboard():
    total_transactions = fraud_data.shape[0]
    total_frauds = fraud_data['class'].sum()
    fraud_percentage = (total_frauds / total_transactions) * 100

    summary_boxes = [
        html.Div(f'Total Transactions: {total_transactions}'),
        html.Div(f'Total Frauds: {total_frauds}'),
        html.Div(f'Fraud Percentage: {fraud_percentage:.2f}%'),
    ]

    # Line chart of fraud trends over time
    fraud_data['purchase_time'] = pd.to_datetime(fraud_data['purchase_time'])
    fraud_trends = fraud_data.groupby(fraud_data['purchase_time'].dt.date)['class'].sum()
    fraud_trends_fig = px.line(fraud_trends, x=fraud_trends.index, y=fraud_trends.values,
                                labels={'x': 'Date', 'y': 'Fraud Cases'},
                                title='Fraud Cases Over Time')

    return summary_boxes, fraud_trends_fig

if __name__ == '__main__':
    app.run_server(debug=True)
"""

with open("dashboard.py", "w") as f:
    f.write(dashboard_code)

print("Dashboard implementation is complete. You can run it using 'python dashboard.py'.")


Dashboard implementation is complete. You can run it using 'python dashboard.py'.
